In [54]:
import chess.gaviota
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [55]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb
%run 11_integer_management.ipynb
%run 12_mirroring.ipynb

# Überprüfen der $S_n$ Mengen

> Potentielles TODO: Noch die anderen Tabellen von Python-Chess als Referenz / Vergleich aufführen

Bei der Bestimmung der $S_n$ Mengen wurden alle möglichen Züge rückwärts durchgeführt.
Damit die Retrograde Analyse erfolgreich zum Gewinnen von Endspielsituationen angewandt werden kann, muss diese Berechnung erfolgreich gewesen sein.
Um sicherzustellen, dass die Situationen, die z. B. in $S_{10}$ zu finden sind, auch wirklich in $<=10$ Zügen beendet werden, wurden in diesem Notebook Funktionen definiert, die diesen Aspekt überprüfen. Hierfür werden zuerst die berechneten $S_n$ Mengen aus der Datei geladen.

In [66]:
%%time
g_piece_list, g_s_n_sequence = load_data("b_b_new_mirror")
g_s_n_sequence = gen_all_integers(g_s_n_sequence, g_piece_list)

Wall time: 2.45 s


Die erste Funktion zur Überprüfung, ist ``fen_in_lower_sequence``. Diese überprüft für eine mitgegebene ``fen``, ob diese in einer $S_n$ Menge liegt, für die gilt: 
$$
n < sequence\_index
$$ 

Die Funktion gibt hierfür einen booleschen Wert zurück.

In [57]:
def int_in_lower_sequence(board_int, sequence, sequence_index):
    for s in sequence[:sequence_index]:
        if board_int in s:
            return True
    return False

Eine weitere Überprüfung liegt darin, dass keine der Schachbretter in eine zu niedrige $S_n$ Menge eingeordnet wurde. Dies wird mit der Funktion ``every_move_of_sequence_in_lower`` umgesetzt. Hierfür erhält sie eine ``sequence_index``, die überprüft werden soll. Für jede Spielsituation wird Folgendes überprüft:

$$
\forall board \in S_n : move \in board.legal\_moves \implies board.push(move) \in S_m \land m<n
$$

Als Ergebnis gibt die Funktion eine boolesche Variable zurück, die ``True`` zurückgibt, falls die Bedingung erfüllt ist, ansonsten ``False``.

In [58]:
def every_move_of_sequence_in_lower(sequence_index, s_n_sequence):
    print("Checking S_" + str(sequence_index) + "...")
    n_integers = s_n_sequence[sequence_index]
    cur_int = 0
    for board_int in n_integers:
        cur_board = to_board(board_int, g_piece_list)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_int = to_integer(cur_board, g_piece_list)
                cur_board.pop()
                if not int_in_lower_sequence(cur_int, s_n_sequence, sequence_index):
                    print("Int: " + str(cur_int) + " not in S lower than " + str(sequence_index))
                    return False
                
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                try:
                    cur_int = to_integer(cur_board, g_piece_list)
                    if int_in_lower_sequence(cur_int, s_n_sequence, sequence_index):
                        in_lower = True
                except ValueError:
                    # Catch wrong pawn conversions
                    pass
                cur_board.pop()
            if not in_lower:
                print(cur_board)
                print(to_integer(cur_board, g_piece_list))
                print("Cannot reach lower S_n")
                return False
    return True

Die folgende Zelle führt diesen Test durch.

In [59]:
for i in range(len(g_s_n_sequence)):
    every_move_of_sequence_in_lower(i, g_s_n_sequence)

Checking S_0...
Checking S_1...
Checking S_2...
Checking S_3...
Checking S_4...
Checking S_5...
Checking S_6...
Checking S_7...
Checking S_8...
Checking S_9...
Checking S_10...
Checking S_11...
Checking S_12...


## Syzygy

Zusätzlich zum rechnerischen Überprüfen der Ergebnisse kann eine andere Methode zum Verifizieren verwendet werden. 
Schach-Endspieldatenbanken sind keine neue Erfindung und wurden bereits von anderen Forschern entwickelt.
Ein lang anhaltendes Projekt, welches gegenwärtig Datenbanken für Situationen mit bis zu 7 Figuren auf dem Spielfeld anbietet ist [Syzygy](https://syzygy-tables.info/).
Das Projekt hält zwei Datentypen für alle Spielsituationen vor:
WDL Daten und DTZ Daten.

WDL steht für Win / Draw / Loss und gibt dem Nutzer eine Information über den Wert einer Spielsituation. 
Eine Anfrage an die Datenbank mit einer Situation wird mit einem der folgenden Werte beantwortet: -2, -1, 0, 1, 2.
Positive Werte implizieren, dass bei perfektem Spiel der aktuelle Spieler gewinnt, negative Werte bedeuten, dass der aktuelle Spieler verliert.
Die 0 bedeutet, dass das Spiel (wenn beide Seiten perfekt spielen) in einem Unentschieden endet.
Eine Zwei ist ein sicherer Sieg / Verlust, während eine Eins in einem Gewinn oder unentschieden mittels der 50-Zug Regel enden kann.

Die Interessantere der Dateien ist die DTZ-Datei.
DTZ steht für Distance to Zero. Die DTZ Tabelle enthält Werte von -100 bis 100. 
Positiv, Negativ und Null kann genau wie WDL interpretiert werden.
Die Zahlen von -100 bis -1 und 1 bis 100 geben die Anzahl der Halbzüge bis zu einem Gewinn (oder Reset der 50-Züge Regel) an.
Stetiges verringern einer positiven DTZ führt also zu einem Gewinn.

Die DTZ-Zahl einer Spielsituation kann mit dem $n$ verglichen werden, in welcher Menge $S_n$ diese Situation in der `.chessAI` eingeordnet wurde.
Stimmen diese Zahlen überein, war die Berechnung korrekt.

*Hinweis: Je nach Ausführung der Syzygy-Tabellen werden halbe oder ganze Züge gespeichert, es muss daher beim Vergleich eine Toleranz von einem $n$ akzeptiert werden.*    

Die Funktion `compare_with_syzygy()` führt den Vergleich zwischen der Endspieldaten und Syzygy durch und gibt die Anzahl falsch eingeordneter Situationen zurück.
Sie erhält dafür ein Objekt vom Type `chess.syzygy` und die Liste von $S_n$-Mengen `s_n_sequence`.
Als Ergebnis, gibt sie die Anzahl der falschen Situationen `total_count` zurück. 

In [60]:
def compare_with_syzygy(syzygy):
    total_count = 0
    
    for n in range(len(g_s_n_sequence)):
        n_count = 0
        for int in g_s_n_sequence[n]:
            chess_board = to_board(int, g_piece_list)
            if n != abs(syzygy.probe_dtz(chess_board)) != n + 1:
                n_count += 1
        print(f"S{n}: Syzygy believes {n_count} of {len(g_s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        total_count += n_count
    print(f"Syzygy believes {total_count} Situations are wrongly placed in the sequence.")
    return total_count

Die Syzygy-Dateien müssen sich entweder im Ordner `./syzygy` befinden oder der Pfad angepasst werden.
Die folgende Zelle initialisiert das Objekt.

In [61]:
syzygy = chess.syzygy.Tablebase()
syzygy.add_directory("./syzygy")

290

Durch einen Funktionsaufruf von `compare_with_syzygy` kann der Vergleich gestartet werden.

In [62]:
compare_with_syzygy(syzygy)

S0: Syzygy believes 0 of 1552 Situations are wrongly placed in the sequence.
S1: Syzygy believes 0 of 6312 Situations are wrongly placed in the sequence.
S2: Syzygy believes 1072 of 1072 Situations are wrongly placed in the sequence.


KeyboardInterrupt: 

## Gaviota

Als Alternative für Syzygy gibt es die Gaviota Endspieltabellen.
Diese können nach ihrer DTM Information ausgewertet werden. DTM steht hierbei für Depth to Mate und gibt an, wie viele Halbzüge eine Situation von einem Schach-Matt entfernt ist.
Ist die Situation unentschieden, ist die DTM Null. 
Für den Vergleich der KI mit Gaviota wird die Funktion `compare_with_gaviota` definiert.
Diese erhält als Parameter ein Objekt vom Typ `chess.Gaviota` und führt den Vergleich mit der globalen Liste von $S_n$ Mengen `g_s_n_sequence` durch.
Sie gibt in der Konsole eine Auswertung aus, und liefert die Anzahl falsch zugeordneter Boards als Rückgabewert zurück.

In [63]:
def compare_with_gaviota(gaviota):
    total_count = 0
    
    for n in range(len(g_s_n_sequence)):
        n_count = 0
        for int_rep in g_s_n_sequence[n]:
            chess_board = to_board(int_rep, g_piece_list)
            if n != abs(gaviota.probe_dtm(chess_board)):
                n_count += 1
                if False:
                    print("Wrong match in S" + str(n) + " - Gaviota says: " + str(abs(gaviota.probe_dtm(chess_board))))
                    print(str(int_rep) + " - " + chess_board.fen())
                    print(chess_board)
        print(f"S{n}: Gaviota believes {n_count} of {len(g_s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        total_count += n_count
    print(f"Gaviota believes {total_count} Situations are wrongly placed in the sequence.")
    return total_count


Die Gaviota-Dateien müssen sich entweder im Ordner `./gaviota` befinden oder der Pfad angepasst werden.
Die folgende Zelle initialisiert das Objekt.

In [64]:
gaviota = chess.gaviota.open_tablebase("./gaviota")

Durch einen Funktionsaufruf von ``compare_with_gaviota`` kann der Vergleich gestartet werden

In [67]:
compare_with_gaviota(gaviota)

S0: Gaviota believes 0 of 1552 Situations are wrongly placed in the sequence.
S1: Gaviota believes 0 of 6312 Situations are wrongly placed in the sequence.
S2: Gaviota believes 0 of 1072 Situations are wrongly placed in the sequence.
S3: Gaviota believes 0 of 2608 Situations are wrongly placed in the sequence.
S4: Gaviota believes 0 of 2344 Situations are wrongly placed in the sequence.
S5: Gaviota believes 0 of 14936 Situations are wrongly placed in the sequence.
S6: Gaviota believes 0 of 3432 Situations are wrongly placed in the sequence.
S7: Gaviota believes 0 of 18432 Situations are wrongly placed in the sequence.
S8: Gaviota believes 0 of 9336 Situations are wrongly placed in the sequence.
S9: Gaviota believes 0 of 30672 Situations are wrongly placed in the sequence.
S10: Gaviota believes 0 of 9320 Situations are wrongly placed in the sequence.
S11: Gaviota believes 0 of 37512 Situations are wrongly placed in the sequence.
S12: Gaviota believes 0 of 21000 Situations are wrongly pl

0

## Matching Rate
Eine weitere Betrachtung, welche über die Qualität der berechneten $S_n$ Mengen Auskunft gibt, ist der Anteil der Situationen, welche abgedeckt werden.
Für diesen Vergleich werden alle validen Stellungen mit den angegebenen Figuren betrachtet, und überprüft wie viele davon sich in einer $S_n$ Menge befinden.
Da nicht alle Stellungen zu einem Sieg geführt werden können, werden die zuvor erklärten Gaviota-Tabellen verwendet, um unentschiedene Situationen herauszufiltern.
Diese werden für den Anteil nicht beachtet.

Zunächst werden globale Variablen für den Vergleich definiert.

In [ ]:
g_total = 0
g_not_matched = 0
g_all_boards = set()

Anschließend wird die Funktion `fill_boards_with_piece`, welche bereits im Notebook `13_calculation_backend.ipynb` erklärt wurde angepasst.
Diese Version fügt für Stellungen, welche alle Figuren beinhalten, einen Vergleich durch, ob sich die Integer-Repräsentation in der globalen Variable `g_all_boards` befindet.
Diese enthält alle Stellungen.
Ist dies nicht der Fall, wird überprüft, ob Gaviota einen DTM-Wert, welcher nicht 0 ist liefert.
Ist dies der Fall, zählt das Board als falsch nicht eingeordnet. 

In [ ]:
def fill_boards_with_piece(int_boards_set, piece, cur_piece_list, total, not_matched):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(g_piece_list)
    for int_board in int_boards_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    continue

                if len(used_squares) + 1 < piece_count: #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
                    continue
                
                total += 1
                if board_int not in g_all_boards:
                    if gaviota.probe_dtm(board_object) != 0: #0 is a draw and not thus relevant for this check
                        if False:
                            print("Board not matched:")
                            print(str(board_int) + " - " + board_object.fen())
                            print(board_object)
                        not_matched += 1
                    
    return finished_boards, total, not_matched


Die Funktion `matchrate_for_player` bestimmt die Abweichungen für den im Parameter `turn` übergebenen Spieler.
Da eine Stellung sich unterscheidet, abhängig welcher Spieler am Zug ist, müssen diese Rechnungen für beide Spieler durchgeführt werden.
Die Funktion funktioniert, wie die bereits in `13_calculation_backend.ipynb` erklärte Funktion `calculate_s0`.

In [ ]:
def matchrate_for_player(turn):
    global g_piece_list, g_total, g_not_matched

    boards = set()
    empty_board = chess.Board(None)
    empty_board.turn = turn
    tmp_piece_list = []
    boards.add(to_integer(empty_board, tmp_piece_list))
    
    total = 0
    not_matched = 0
   
    for piece in g_piece_list:
        boards, total, not_matched = fill_boards_with_piece(boards, piece, tmp_piece_list, total, not_matched)
        tmp_piece_list.append(piece)
    
    g_total += total
    g_not_matched += not_matched

Die Funktion `matchrate` führt die Rechnung für beide Spieler durch.

In [ ]:
def matchrate():
    matchrate_for_player(chess.WHITE)
    matchrate_for_player(chess.BLACK)
    percentage = (g_total - g_not_matched) / g_total * 100
    print("Matching Rate:" + str(percentage) + "%")

Die folgende Zelle füllt die Variable `g_all_boards` und führt die Rechnung durch. 

In [ ]:
g_all_boards = set.union(*g_s_n_sequence)
matchrate()


## Ergebnis der Validierung

Durch Nutzung dieses Notebooks kann festgestellt werden, dass die Enspieldaten für drei Figuren (König, König und Dame sowie König, König und Turm) optimal sind.
Es werden 100% der gewinnbaren Situationen abgedeckt (`matching rate`) und diese mit derselben Genauigkeit, wie die Endspieldaten von Syzygy und Gaviota.
Dasselbe gilt auch für die Situationen mit König, König, Läufer und Springer.

Die Validierung liefert von der Erwartung abweichende Ergebnisse für die Situation König und zwei Läufer gegen König.
Die Abdeckung der gewinnbaren Situationen liegt in 40 $S_n$ Mengen bei 98,98%. Das bedeutet, dass ca. 1680 valide und gewinnbare Situationen nicht durch die KI gelöst werden können.
Weiter zeigen die anderen Tests, dass Situationen im Vergleich zu Gaviota in einem zu hohem $S_n$ eingeordnet wurden. Da es nur 40 Mengen gibt, werden maximal 20 Züge bis zu einem Sieg durchgeführt. Es können also auch unter Betrachtung der 50-Züge Regel alle Situationen zu einem Sieg geführt werden.
Auch der erste Test schlägt fehl. Dies führt Ebenfalls zu Ineffizienzen beim Spiel gegen die KI.
Eine weitere Besonderheit lässt sich beim Vergleich des Syzygy Tests mit dem Gaviota Test erkennen. Ab $S_2$ betrachtet Syzygy alle Situationen als falsch eingeordnet. Da Situationen aus $S_3$ in drei Halbzügen zum Sieg geführt werden können, ist diese Information offensichtlich Falsch.
TODO: Beispiel
Dieses Phänomen wurde nicht genauer untersucht und stattdessen der Vergleich mit Gaviota bevorzugt. 

Im Rahmen der Optimierung und Fehlersuche wird festgestellt, dass eine Berechnung mit den Spiegelungen deaktiviert keine falschen Zuordnungen für die Situationen mit König gegen König und zwei Läufer liefert.
Hieraus folgt die Vermutung, dass mit dem Code für Spiegelungen ein Problem vorliegt. Um diese These zu überprüfen, wird der Spiegelungs-Code angepasst, die Funktionen der `chess` Bibliothek für die horizontale, vertikale und diagonale Spiegelung zu verwenden. Die Bibliothek bietet keine Funktionen für die Rotation von Situationen.
Als Ergebnis dieses Tests wurde festgestellt, das weniger aber weiterhin falsche Zuordnungen stattfinden. Es wird daher die These aufgestellt, dass die Verwendung von Spiegelungen eine negative Auswirkung auf die Ergebnisse hat. Der Grund hierfür ist nicht bekannt. Da die Performance geringer ist, werden die Spiegelungsfunktionen der Bibliothek nicht verwendet.    

Der Sonderfall König, König Bauer, welcher als Einziger eine Änderung der Figuren auf dem Spielfeld enthält, kann von diesem Notebook nicht komplett validiert werden.
Da die Spielzüge nach einer Wandlung des Bauerns zur Dame optimal berechnet sind, wurde dieses Notebook nicht angepasst, um die Schritte des Bauerns bis zur Umwandlung zu validieren.


In [ ]:
b = chess.Board("8/6B1/8/8/2B5/6K1/8/6k1 w - - 0 1")
print(b)
print(b.piece_map())
b.set_piece_map({26: chess.Piece.from_symbol('B'), 54: chess.Piece.from_symbol('B'), 22: chess.Piece.from_symbol('K'), 6: chess.Piece.from_symbol('k')})
print(b.piece_map())
